


# Connect to drive

In [86]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Prereq

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.5 MB/s eta 0:00:00


In [35]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=ae3632c54778fef78286fa04e1a3e9d02894984e50f82400a66fe723f0680886
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [3]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
import torch.nn as nn

In [87]:
from sklearn.metrics.pairwise import cosine_similarity

In [88]:
import pandas as pd

# Read

In [57]:
altex_reviews_path = '/content/drive/MyDrive/Colab Notebooks/sw/all_reviews_instances_altex.txt'
decathlon_reviews_path = '/content/drive/MyDrive/Colab Notebooks/sw/all_reviews_instances_decathlon.txt'
emag_reviews_path = '/content/drive/MyDrive/Colab Notebooks/sw/all_reviews_instances_emag.txt'

:Review312747_1 rdf:type owl:NamedIndividual , rev:Review ; rev:text "Raport preț excelent , recomand cu incredere" ; :hasAuthor "Noge Catalin" ; :hasRating 4 ; :aggregatesOpinion marl:Neutral .

In [58]:
content = []
with open(altex_reviews_path,'r',encoding='utf-8') as f:
  altex = f.readlines()
with open(decathlon_reviews_path,'r',encoding='utf-8') as f:
  decathlon = f.readlines()
with open(emag_reviews_path,'r',encoding='utf-8') as f:
  emag = f.readlines()

content.extend(altex)
content.extend(decathlon)
content.extend(emag)

In [60]:
len(content)

20732

In [59]:
result = []
for line in content:
  if str(line).startswith(":Review"):
    nod = line.split("rdf:type")[0]
    text = line.split("rev:text \"")[1]
    text = text.split("\" ;")[0]
    result.append((nod,text))

In [61]:
result

[(':Review312747_1 ', 'Raport preț excelent , recomand cu incredere'),
 (':Review267885_1 ', 'Bun, se conecteaza usor, materiale de calitate'),
 (':Review267885_2 ',
  'un volan decent pentru pretul lui. nu dezamageste ! Nu il poti compara cu g29, dar si pretul e la  sfert fata de acesta. Cumparati-l cu incredere ! '),
 (':Review267885_3 ',
  'L-am cumparat de nou stricat, cei de la magazinul de unde l-am cumparat nu au vrut sa imi dea retur pe loc si banii inapoi, nu-i corect'),
 (':Review267885_4 ',
  'Sincer am fost incantat de el primele 2 zile,dupa am observat ca are deadzone cand viram usor stanga,iar cand viram dreapta vira ok.Am trecut peste zicand ca am dat putin pe el,in 900 de grade nu se simtea asa rau,doar cand mergeai cu viteza si aveai nevoie de acuratete,ce m-a nemultumit a fost ca in 270 de grade a fost insuportabil deoarece aveam nevoie de unghiuri precise,volanul mergand sacadat(tin sa precizez ca nu este o miscare cursiva pe ecran,el se misca sacadat,in prezent i-am

In [64]:
df = pd.DataFrame(result,columns=['nod','text'])
df.drop_duplicates(subset=['nod'],inplace=True)
df  

,nod,text
0,:Review312747_1,"Raport preț excelent , recomand cu incredere"
1,:Review267885_1,"Bun, se conecteaza usor, materiale de calitate"
2,:Review267885_2,un volan decent pentru pretul lui. nu dezamage...
3,:Review267885_3,"L-am cumparat de nou stricat, cei de la magazi..."
4,:Review267885_4,"Sincer am fost incantat de el primele 2 zile,d..."
...,...,...
10303,:Review24997179_37,"Sunt ok, le-am primit după două zile de când a..."
10304,:Review24997179_38,"La pretul de 160 lei, sunt foarte multumit de ..."
10305,:Review24997179_39,Pe acelasi pret?!!?<br /> Un pic prea mult bas...
10306,:Review24997179_40,SE aude foarte bine


# Sentence Transformers

https://towardsdatascience.com/bert-for-measuring-text-similarity-eec91c6bf9e1

In [36]:
from sentence_transformers import SentenceTransformer

In [37]:
model_st = SentenceTransformer('bert-base-nli-mean-tokens')

In [65]:
sentences = df['text']
sentence_embeddings = model_st.encode(sentences)

In [68]:
sentence_embeddings.shape

(10308, 768)

In [70]:
sentence_embeddings[0][0]

0.010494284

In [ ]:
import numpy as np
nparray_s = np.array(sentence_embeddings)

In [75]:
df['embedding'] = ''

In [76]:
df.head()

,nod,text,embedding
0,:Review312747_1,"Raport preț excelent , recomand cu incredere",
1,:Review267885_1,"Bun, se conecteaza usor, materiale de calitate",
2,:Review267885_2,un volan decent pentru pretul lui. nu dezamage...,
3,:Review267885_3,"L-am cumparat de nou stricat, cei de la magazi...",
4,:Review267885_4,"Sincer am fost incantat de el primele 2 zile,d...",


In [77]:
for idx,row in df.iterrows():
  row['embedding'] = nparray_s[idx]

In [78]:
df.head()

,nod,text,embedding
0,:Review312747_1,"Raport preț excelent , recomand cu incredere","[0.010494284, 0.24036112, 1.4413524, 0.4629804..."
1,:Review267885_1,"Bun, se conecteaza usor, materiale de calitate","[0.0845401, 0.07536337, 1.6805738, 0.38317433,..."
2,:Review267885_2,un volan decent pentru pretul lui. nu dezamage...,"[-0.12203061, 0.40780303, 1.3255798, 0.5204835..."
3,:Review267885_3,"L-am cumparat de nou stricat, cei de la magazi...","[-0.11117437, 0.253098, 0.9758826, 0.29502293,..."
4,:Review267885_4,"Sincer am fost incantat de el primele 2 zile,d...","[-0.24998382, 0.92990744, 0.46528113, 0.537898..."


In [79]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/sw/rev_with_embeddings.csv')

# Get similarity

In [81]:
cosine_similarity(
    [nparray_s[0]],
    nparray_s[1:])

array([[0.8014158 , 0.7350025 , 0.783408  , ..., 0.47123262, 0.79193723,
        0.5745644 ]], dtype=float32)

# Create triplets

In [89]:
path = '/content/drive/MyDrive/Colab Notebooks/sw/rev_with_embeddings.csv'
df = pd.read_csv(path)

In [90]:
df.head()

,Unnamed: 0,nod,text,embedding
0,0,:Review312747_1,"Raport preț excelent , recomand cu incredere",[ 1.04942843e-02 2.40361124e-01 1.44135237e+...
1,1,:Review267885_1,"Bun, se conecteaza usor, materiale de calitate",[ 8.45400989e-02 7.53633678e-02 1.68057382e+...
2,2,:Review267885_2,un volan decent pentru pretul lui. nu dezamage...,[-1.22030608e-01 4.07803029e-01 1.32557976e+...
3,3,:Review267885_3,"L-am cumparat de nou stricat, cei de la magazi...",[-1.11174367e-01 2.53098011e-01 9.75882590e-...
4,4,:Review267885_4,"Sincer am fost incantat de el primele 2 zile,d...",[-2.49983817e-01 9.29907441e-01 4.65281129e-...


In [91]:
template = f""":hasEmbedding rdf:type owl:ObjectProperty ;
               rdfs:domain rev:Review .\n"""
for idx,row in df.iterrows():
  nod = row['nod']
  emb = row['embedding']
  template += f"""{nod} :hasEmbedding \"{emb} \"^^xsd:string . \n"""
template

In [92]:
template

In [93]:
with open('triplets.txt','w',encoding='utf-8') as f:
  f.write(template)